In [1]:
#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the list of edges of a Veblen hypergraph V
#output: the flattening of V
def flat(V):
    L=[]
    for element in V:
        L.append(tuple(element))
    W=Set(L)
    return sorted(tuple(W))

#in: uniformity k, number of edges d, number of vertices n
#out: the list of Veblen hypergraphs with uniformity k on d edges and \leq n vertices
def Veblen(k,d,n):
    lst=[]
    for element in list(hypergraphs.nauty(d, n, uniform=k, multiple_sets=True)):
        M=edges_to_matrix(element)
        vmod = [x%k for x in M*vector([1]*M.ncols())]
        v = [x for x in M*vector([1]*M.ncols())]
        if sum(vmod) == 0 and (0 not in v):
            lst.append(tuple(sorted(element)))
    return(lst)

In [ ]:
#Written by Dr. Alex Duncan 
#Depends on the invariance of the resultant under “change of base” or ring homomorphisms of the coefficient ring.

import time
import datetime

# This is more direct, but probably slower.
def uniform_hypergraph_charpoly_naive( n, edges):
    d = len(edges[0])
    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)
    t_poly = f"t*({fermat}) - {d}*({edge_poly})"

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[t][{variable_names}]')
    F = macaulay2(t_poly)
    disc = F.discriminant()
    return disc.sage()

# This computes the polynomial by multiple evaluation
# followed by Lagrange interpolation
# (maybe faster for larger examples?)
# It gives an estimated time remaining counter.
def uniform_hypergraph_charpoly( n, edges, estimate_time = True):
    d = len(edges[0])

    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[{variable_names}]')

    total_degree = n*(d-1)^(n-1)
    points = []
    start_time = time.time()
    for t in range(total_degree+1):
        t_poly = f"{t}*({fermat}) -  {d}*({edge_poly})"
        F = macaulay2(t_poly)
        disc = F.discriminant()
        points.append([t,disc.sage()])
        if estimate_time: # totally optional
            time_passed = time.time() - start_time
            time_remaining = time_passed*(total_degree-t)/(t+1)
            print(f"Time Remaining: {datetime.timedelta(seconds=time_remaining)}")
    R.<t> = PolynomialRing(QQ)
    return R.lagrange_polynomial(points)

# t=var("t")
# n=4
# edges = [[0,1,2],[0,1,3],[0,2,3],[1,2,3]]

# poly = uniform_hypergraph_charpoly(n,edges, estimate_time = False)
# f=expand(factor(poly))
# f = f/f.leading_coefficient()
# print(f)